In [1]:
# Loading Model
from Utils import *
model = CapsNet(input_shape=(20,6000,3), n_class=3, routings=3)
model.load_weights('best_model_CCTCAPS4.h5')
model.summary()

2023-04-05 18:54:45.959861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 18:54:46.481139: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9336 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5


KerasTensor(type_spec=TensorSpec(shape=(None, 1880, 40), dtype=tf.float32, name=None), name='layer_normalization_4/batchnorm/add_1:0', description="created by layer 'layer_normalization_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 20, 94, 40), dtype=tf.float32, name=None), name='reshape/Reshape:0', description="created by layer 'reshape'")
Instructions for updating:
Use fn_output_signature instead
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 6000, 3  0           []                               
                                )]                                                                
                                                                                                  
 cct_tokenizer1 (CCTTokenizer1)  (None, 1880, 40)    73080       ['input_1[0][0]'] 

 dropout_4 (Dropout)            (None, 1920, 4)      0           ['primarycap_squash[0][0]']      
                                                                                                  
 digitcaps (CapsuleLayer)       (None, 3, 6)         138240      ['dropout_4[0][0]']              
                                                                                                  
 capsnet (Length)               (None, 3)            0           ['digitcaps[0][0]']              
                                                                                                  
Total params: 260,456
Trainable params: 260,456
Non-trainable params: 0
__________________________________________________________________________________________________


In [2]:
# Set qml and data paths
qmlpath = r'/run/media/omar/My Passport1/data2022/events/'
datpath = r'/run/media/omar/My Passport/WaveformsTexas/waveforms/'

#load eventID
catnameTest = np.load('TEST_SWD.npy')


In [3]:
from obspy.core.event import read_events
import numpy as np
import glob
import obspy
import obspy
import numpy as np
from obspy.signal.tf_misfit import cwt
from scipy import signal


fs = 100
dt = 1/100
f_min = 1
f_max = 45


catnameTest = np.array(catnameTest)
outfinSW = []
outfinNW = []

for iu in range(len(catnameTest)):
    print(iu)
    pa = (qmlpath + catnameTest[iu] + '.qml')
    pa1 = (datpath + catnameTest[iu] + '.mseed')
    cat = read_events(pa)
    st = obspy.read(pa1)
    st = st.resample(100)
    st = st.merge(fill_value=0)
    st = st.filter(type='bandpass', freqmin = 1.0, freqmax = 45, corners=2, zerophase=True)
    st = st.taper(max_percentage=0.001, type='cosine', max_length=2) 

    datfin = []

    for i in cat[0].picks:
        dat = []
        if i.phase_hint=='P':
            st2 = st.copy()
            st2 = st2.select(station=i.waveform_id.station_code)
            if len(st2)==3:
                if len(st2[2].data)>=6000:
                    t = obspy.UTCDateTime(i.time)
                    stall = st2.trim(t-1,t+59-0.01)
                    #print(stall)
                    a0 = stall[0].data
                    a1 = stall[1].data
                    a2 = stall[2].data
                    if (len(a0)==6000) and (len(a1)==6000) and (len(a2)==6000):
                        dat.append(a0)
                        dat.append(a1)
                        dat.append(a2)

                if (len(a0)==6000):
                    dat = np.array(dat) 
                    temp0 = cwt(a0, dt, 10, f_min, f_max, nf=20,  wl='morlet')
                    temp0 = np.clip(np.abs(temp0)[-1::-1], 0, 100)
                    #temp0 = temp0[3:,:]
                    ma = np.max(np.abs(temp0))
                    if ma==0:
                        ma = 1
                    temp0 = temp0 / ma


                    temp1 = cwt(a1, dt, 10, f_min, f_max, nf=20,  wl='morlet')
                    temp1 = np.clip(np.abs(temp1)[-1::-1], 0, 100)
                    #temp1 = temp1[3:,:]
                    ma = np.max(np.abs(temp1))
                    if ma==0:
                        ma = 1
                    temp1 = temp1 / ma

                    temp2 = cwt(a2, dt, 10, f_min, f_max, nf=20,  wl='morlet')
                    temp2 = np.clip(np.abs(temp2)[-1::-1], 0, 100)
                    #temp2 = temp2[3:,:]
                    ma = np.max(np.abs(temp2))
                    if ma==0:
                        ma = 1
                    temp2 = temp2 / ma

                    tmp = np.zeros((20,6000,3))
                    tmp[:,:,0] = np.abs(temp0[:,0:6000])
                    tmp[:,:,1] = np.abs(temp1[:,0:6000])
                    tmp[:,:,2] = np.abs(temp2[:,0:6000])                    
                    datfin.append(tmp) 
    datfin = np.array(datfin)                
    out = model.predict(datfin, batch_size=16)
    out = np.argmax(out,axis=-1)
    outfinSW.append(out)
    outfinNW.append(np.mean(out))

0


2023-04-05 18:57:16.814156: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2023-04-05 18:57:18.304645: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2023-04-05 18:57:18.361245: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1/1 [==============================] - 5s 5s/step
1
1/1 [==============================] - 0s 266ms/step
2
1/1 [==============================] - 0s 319ms/step
3
1/1 [==============================] - 0s 373ms/step
4
1/1 [==============================] - 0s 67ms/step
5
2/2 [==============================] - 0s 206ms/step
6
1/1 [==============================] - 0s 224ms/step
7
2/2 [==============================] - 0s 24ms/step
8
1/1 [==============================] - 0s 57ms/step
9
1/1 [==============================] - 0s 333ms/step
10
1/1 [==============================] - 0s 66ms/step
11
2/2 [==============================] - 0s 123ms/step
12
2/2 [==============================] - 0s 165ms/step
13
2/2 [==============================] - 0s 17ms/step
14
1/1 [==============================] - 0s 242ms/step
15
2/2 [==============================] - 0s 210ms/step
16
1/1 [==============================] - 0s 370ms/step
17
1/1 [==============================] - 0s 68ms/step
18
2/2 [=====

In [4]:
# The ouput is zero for HF, 1 for SWD, 2 for SWDpd
# outfinNW ---> Output for event-based.
# outfinSW ---> Output from Single-station.
outfinw = np.round(outfinNW)
outfinsw = np.concatenate( outfinSW, axis=0 )
